In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import csv
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import h5py
from torchvision import transforms
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
def imageGenerator():
    input_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\data'
    output_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\converted_data'
    datas = []
    img_size = (128,128)
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.mat'):
            file_path = os.path.join(input_folder,file_name)
            
            with h5py.File(file_path, 'r') as f:
                image = np.array(f['cjdata/image']).T
                label = int(np.array(f['cjdata/label'])[0,0])
                tumor_border = np.array(f['cjdata/tumorBorder']).T
                tumor_mask = np.array(f['cjdata/tumorMask']).T
    
            image = image.astype(np.float64)
            im_norm = 255*(image-image.min())/(image.max()-image.min())
            im_uint8 = im_norm.astype(np.uint8)
            im_pil = Image.fromarray(im_uint8)
            im_pil = im_pil.resize(img_size)

            tumorMask = tumor_mask.astype(np.float64)
            tm_norm = 255*(tumor_mask-tumor_mask.min())/(tumor_mask.max()-tumor_mask.min())
            tm_uint8 = tm_norm.astype(np.uint8)
            tm_pil = Image.fromarray(tm_uint8)
            tm_pil = tm_pil.resize(img_size)
            
            datas.append((np.array(im_pil,dtype=np.float32),np.array(tm_pil,dtype=np.float32)))


    return datas

In [3]:
try:
    datas = imageGenerator()
except ValueError:
    print('Hibás érték')

FileNotFoundError: [WinError 3] A rendszer nem találja a megadott elérési utat: '\\data'

In [ ]:
#datas[0][2] - [sor][oszlop]

In [ ]:
labels = [ 'Nothing', 'meningioma', 'glioma', 'pituitary tumor']

In [ ]:
images = [item[0] for item in datas]
tumorMask = [item[1] for item in datas]

In [ ]:
from sklearn.model_selection import train_test_split
i_train, i_test, tm_train, tm_test = train_test_split(images, tumorMask, test_size=0.2, random_state=42)

In [ ]:
def double_conv_block(x, n_filters):
    x = layers.Conv2D(n_filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = layers.Conv2D(n_filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p

def upsample_block(x, conv_features, n_filters):
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding='same')(x)
    x = layers.concatenate([x, conv_features])
    x = layers.Dropout(0.3)(x)
    x = double_conv_block(x, n_filters)
    return x

In [ ]:
def unet_model(output_channels):
    inputs = layers.Input(shape=(128, 128, 1))
    
    f1, p1 = downsample_block(inputs, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)
    
    intermediate_block = double_conv_block(p4, 1024)

    u6 = upsample_block(intermediate_block, f4, 512)
    u7 = upsample_block(u6, f3, 256)
    u8 = upsample_block(u7, f2, 128)
    u9 = upsample_block(u8, f1, 64)
    
    outputs = layers.Conv2D(output_channels, 256, padding='same', activation='softmax')(u9)
    unet_model = tf.keras.Model(inputs, outputs, name='U-Net')
    
    return unet_model

In [ ]:
output_channels = 256
model = unet_model(output_channels)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
i_train = np.array(i_train,dtype=np.uint8)
i_test = np.array(i_test,dtype=np.uint8)
tm_train = np.array(tm_train,dtype=np.uint8)
tm_test = np.array(tm_test,dtype=np.uint8)

In [ ]:
EPOCHS = 6
history = model.fit(i_train,tm_train,batch_size=8, epochs=6, validation_data=(i_test,tm_test))

In [ ]:
#print(np.unique(tm_train))

In [ ]:
#x_train = np.expand_dims(X_train, axis=-1)
#tm_train = np.expand_dims(X_train, axis=-1)

In [ ]:
#tm_train[0]